<a href="https://colab.research.google.com/github/aderylo/movie-genres-prediction/blob/master/movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install contractions
!pip install autocorrect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np 
import re
import pandas as pd 
import nltk 
import contractions
from bs4 import BeautifulSoup
from autocorrect import Speller

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
ss = nltk.SnowballStemmer(language = 'english')
w = [ss.stem(word) for word in ["exciting"]]
print(w)

['excit']


## Geting dataset 
Unfortuently, reading directly a file from a git url wasn't working properly, so the esiest solution was to clone whole repo. 

In [4]:
!git clone https://github.com/aderylo/movie-genres-prediction.git

fatal: destination path 'movie-genres-prediction' already exists and is not an empty directory.


In [5]:
df = pd.read_csv('movie-genres-prediction/mpst_full_data.csv', delimiter=',')
nRow, nCol = df.shape
df.head(5)

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb


## Cleaning data

In [6]:
def strip_html_tags(text):
    """ 
    This function will remove all the occurrences of html tags from the text.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of html tags.
        
    Example:
    Input : This is a nice place to live. <IMG>
    Output : This is a nice place to live.  
    """
    # Initiating BeautifulSoup object soup.
    soup = BeautifulSoup(text, "html.parser")
    # Get all the text other than html tags.
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

In [7]:
def remove_links(text):
    """
    This function will remove all the occurrences of links.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of all types of links.
        
    Example:
    Input : To know more about cats and food & website: catster.com  visit: https://catster.com//how-to-feed-cats
    Output : To know more about cats and food & website: visit:     
    
    """
    
    # Removing all the occurrences of links that starts with https
    remove_https = re.sub(r'http\S+', '', text)
    # Remove all the occurrences of text that ends with .com
    remove_com = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
    return remove_com

In [8]:
def spelling_correction(text):
    ''' 
    This function will correct spellings.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text after corrected spellings.
        
    Example: 
    Input : This is Oberois from Dlhi who came heree to studdy.
    Output : This is Oberoi from Delhi who came here to study.
    '''
    # Check for spellings in English language
    spell = Speller(lang='en')
    Corrected_text = spell(text)
    return Corrected_text

In [9]:
stoplist = nltk.corpus.stopwords.words('english') 
stoplist = set(stoplist)
def removing_stopwords(text):
    """This function will remove stopwords which doesn't add much meaning to a sentence 
       & they can be remove safely without comprimising meaning of the sentence.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text after omitted all stopwords.
        
    Example: 
    Input : This is Kajal from delhi who came here to study.
    Output : ["'This", 'Kajal', 'delhi', 'came', 'study', '.', "'"] 
    
   """
    # repr() function actually gives the precise information about the string
    text = repr(text)
    # Text without stopwords
    No_StopWords = [word for word in nltk.tokenize.word_tokenize(text) if word.lower() not in stoplist ]
    # Convert list of tokens_without_stopwords to String type.
    words_string = ' '.join(No_StopWords)    
    return words_string

In [10]:
def tokenizing(text): 
  tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
  return ' '.join(tokenizer.tokenize(text.lower()))

In [11]:
def clean_text(text): 
  text = strip_html_tags(text)
  text = remove_links(text)
  # text = spelling_correction(text) # for faster execution remove this one
  text = contractions.fix(text)
  text = tokenizing(text)
  text = removing_stopwords(text)
  text = re.sub("\S*\d\S*", "", text).strip()
  return text

#Takes around a minute or so
df['preprocessed_plots'] = df['plot_synopsis'].apply(clean_text)

In [12]:
df['preprocessed_plots']

0        'note synopsis orginal italian release segment...
1        'two thousand years ago nhagruul foul sorcerer...
2        'matuschek gift store budapest workplace alfre...
3        'glenn holland morning person anyone standards...
4        'in may  cuban man named tony montana al pacin...
                               ...                        
14823    'in  russ richards john travolta weatherman ha...
14824    'in russia media covers tony stark disclosure ...
14825    'during north african campaign world war ii ca...
14826    'steven kenet catches unfaithful wife apartmen...
14827    'sometime  chicago man cecil moe michael madse...
Name: preprocessed_plots, Length: 14828, dtype: object

In [13]:
def remove_sapces(text): 
  return text.replace(" ","")

df['tags']=df['tags'].apply(remove_sapces)
df['tags']

0                    cult,horror,gothic,murder,atmospheric
1                                                 violence
2                                                 romantic
3                      inspiring,romantic,stupid,feel-good
4        cruelty,murder,dramatic,cult,violence,atmosphe...
                               ...                        
14823                                        comedy,murder
14824                              goodversusevil,violence
14825                                              antiwar
14826                                               murder
14827                                        christianfilm
Name: tags, Length: 14828, dtype: object

### How well did we clean the data? 

In [14]:
def max_len(x):
    a=x.split()
    return len(a)

v=Tokenizer()
v.fit_on_texts(df['preprocessed_plots'])
vocab_size = len(v.word_index) + 1


print(max(df['preprocessed_plots'].apply(max_len)))
print(vocab_size)

5833
125854


## Encoding before training

In [15]:
train=df.loc[df.split=='train']
test=df.loc[df.split=='test']
cv=df.loc[df.split=="val"]

train=train.reset_index()
test=test.reset_index()
cv=cv.reset_index()


In [16]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(","), binary='true')
y_train = vectorizer.fit_transform(train['tags']).toarray()
y_test = vectorizer.transform(test['tags']).toarray()

print(y_train)
print(vectorizer.inverse_transform(y_train)[0])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]
['atmospheric' 'cult' 'gothic' 'horror' 'murder']


In [17]:
vect=Tokenizer()
vect.fit_on_texts(train['plot_synopsis'])
vocab_size = len(vect.word_index) + 1
print(vocab_size)

122195


In [18]:
encoded_docs_train = vect.texts_to_sequences(train['preprocessed_plots'])
max_length = vocab_size
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=1200, padding='post')
print(padded_docs_train)

[[ 4660 62208  2514 ...     0     0     0]
 [ 4481   143   875 ...     0     0     0]
 [ 4094  2892 28344 ...   140  6946  2001]
 ...
 [62209  2717   539 ...     0     0     0]
 [25350  5118  2731 ...     0     0     0]
 [ 1269  2392  2530 ...     0     0     0]]


In [19]:
encoded_docs_test =  vect.texts_to_sequences(test['preprocessed_plots'])
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=1200, padding='post')
encoded_docs_cv = vect.texts_to_sequences(cv['preprocessed_plots'])
padded_docs_cv = pad_sequences(encoded_docs_cv, maxlen=1200, padding='post')

## Model 

In [20]:
model = keras.Sequential()
# Configuring the parameters
model.add(layers.Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(layers.LSTM(128, return_sequences=True))  
# Adding a dropout layer
model.add(layers.Dropout(0.5))
model.add(layers.LSTM(64))
model.add(layers.Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(layers.Dense(71, activation='sigmoid'))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1200, 50)          6109750   
                                                                 
 lstm (LSTM)                 (None, 1200, 128)         91648     
                                                                 
 dropout (Dropout)           (None, 1200, 128)         0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 71)                4615      
                                                                 
Total params: 6,255,421
Trainable params: 6,255,421
Non-

In [21]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

callbacks = [
    EarlyStopping(monitor='val_loss', patience=4, verbose=0),
]

model.compile(optimizer='adam', loss='binary_crossentropy', metrics = METRICS)

In [ ]:
history = model.fit(padded_docs_train,y_train,
                        epochs = 10,
                        verbose = 1,
                        validation_data=(padded_docs_test, y_test),
                        batch_size=16,
                        callbacks=callbacks)

Epoch 1/10
594/594 [==============================] - 75s 108ms/step - loss: 0.1721 - tp: 1387.0000 - fp: 8239.0000 - tn: 637458.0000 - fn: 26635.0000 - accuracy: 0.9482 - precision: 0.1441 - recall: 0.0495 - auc: 0.7513 - val_loss: 0.1419 - val_tp: 2.0000 - val_fp: 69.0000 - val_tn: 201495.0000 - val_fn: 9020.0000 - val_accuracy: 0.9568 - val_precision: 0.0282 - val_recall: 2.2168e-04 - val_auc: 0.8299
Epoch 2/10
 62/594 [==>...........................] - ETA: 46s - loss: 0.1553 - tp: 83.0000 - fp: 744.0000 - tn: 66854.0000 - fn: 2751.0000 - accuracy: 0.9504 - precision: 0.1004 - recall: 0.0293 - auc: 0.7905

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

def plot_metrics(history):
    metrics =  ['loss', 'auc', 'precision', 'recall']
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2,2,n+1)
        plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
                 color=colors[0], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
            plt.ylim([0.8,1])
        else:
            plt.ylim([0,1])

        plt.legend()

plot_metrics(history)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
predictions=model.predict([padded_docs_test])
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

In [ ]:
def predict_sample():
    t = train.sample(1)
    encoded_docs = vect.texts_to_sequences(t['preprocessed_plots'])
    padded_docs = pad_sequences(encoded_docs, maxlen=1200, padding='post')
    pred = model.predict(padded_docs).tolist()
    for i in range(len(pred[0])):
        if(pred[0][i] < 0.1):
            pred[0][i] = 0
        else:
            pred[0][i] = 1
    
    print("Original tags -->", t['tags'].values)
    print("Predicted tags -->", vectorizer.inverse_transform(pred))

predict_sample()